# Covid-19 New Cases Dashboard: Comparison between Four UK Countries

##### In the following chart we can see and compare the daily growth of new Covid-19 cases among four UK Countries (England, Scotland, Wales, and Northern Ireland). The chart has been made using data published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).




In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate

with open("newCases.json", "rt") as INFILE:
    newCases=json.load(INFILE)


In [4]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=rawdata['data']
    
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()  
    def  parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    
    Englist = [England for England in datalist if England['areaName'] == 'England']
    Scotlist = [Scotland for Scotland in datalist if Scotland['areaName'] == 'Scotland']
    Waleslist = [Wales for Wales in datalist if Wales['areaName'] == 'Wales']
    NorIrelist = [NorIre for NorIre in datalist if NorIre['areaName'] == 'Northern Ireland']
    
    UKdf=pd.DataFrame(index=index, columns=['England', 'Scotland', 'Wales', 'Northern Ireland'])
    
    for entry in Englist:
        date=entry['date']
        UKdf.loc[date, 'England']=entry['newCases']
    for entry in Scotlist:
        date=entry['date']
        UKdf.loc[date, 'Scotland']=entry['newCases']
    for entry in Waleslist:
        date=entry['date']
        UKdf.loc[date, 'Wales']=entry['newCases']
    for entry in NorIrelist:
        date=entry['date']
        UKdf.loc[date, 'Northern Ireland']=entry['newCases']
    

    UKdf.fillna(0.0, inplace=True)
    return UKdf

UKdf=wrangle_data(newCases)


In [5]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=nation'
    ]
    structure = {
        "areaName": "areaName",
        "date": "date",
        "newCases": "newCasesByPublishDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    newCases=api.get_json()
    with open("newCases.json", "wt") as OUTF:
        json.dump(newCases, OUTF)
    with open("newCases.json", "rt") as INFILE:
        newCases=json.load(INFILE)
    return newCases


In [6]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    try:
        newCases=access_api()
        global UKdf
        UKdf=wrangle_data(newCases)
        refresh_graph()
        apibutton.button_style='success'
        apibutton.icon="check"
        print('Success!', end = '\r', flush = True)
    except:
        apibutton.button_style='danger'
        apibutton.icon="warning"
        print('Failed!', end = '\r', flush = True)
        
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='info',
    tooltip='Click to download current Public Health England data',
    icon='download'
)

apibutton.on_click(api_button_callback) 

display(apibutton)


Button(button_style='info', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click t…

In [11]:
nation=wdg.SelectMultiple(
    options=['England', 'Scotland','Wales','Northern Ireland'],
    value=('England', 'Scotland','Wales','Northern Ireland'),
    rows=4,
    description='Nation:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.VBox([nation, scale])

def newCases_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        UKdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

    
def refresh_graph():
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other 
    scale.value=current
  
   
graph=wdg.interactive_output(newCases_graph, {'gcols': nation, 'gscale': scale})

layout = wdg.HBox([graph, controls])

display(layout)

## About the Data

#### England
Data include only pillar 1 cases until 2 July, from when pillar 2 cases are also included. A technical issue identified on 2 October 2020 resulted in a delay in reporting of 15,841 cases between 25 September and 2 October. These cases were instead reported on 3 and 4 October creating an artificial spike in the number of cases reported on these dates. Cases by specimen date data are not affected.

#### Northern Ireland
Data include only pillar 1 cases until 26 June, from when pillar 2 cases are also included.

#### Scotland
Data include only pillar 1 cases until 15 June, from when pillar 2 cases are also included. Scotland's data include a batch of cases for which specimen date was not available over a 10 day period between 15 and 25 April. These samples were assigned a specimen date in the midpoint within this range (20 April) causing the artificial spike in the chart.

#### Wales
Data include mainly pillar 1 cases until 14 July, from when pillar 2 cases have been included. This included 842 historic pillar 2 cases that had not previously been reported in the UK total, causing an increase in the UK total.

# ------------------------------------------------------------------------------------------------------





[Covid-19 New Cases Dashboard: Comparison between Four UK Countries](https://github.com/irsyadqomar/covid19dashboard) (C) Irsyad Qomar, 2020 ([ec20025@qmul.ac.uk](mailto:ec20025@qmul.ac.uk)). All rights reserved.
*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*